In [1]:
import logging

from automata.cli.commands import reconfigure_logging
from automata.config.base import AgentConfigName
from automata.config.openai_agent import OpenAIAutomataAgentConfigBuilder
from automata.agent.providers import OpenAIAutomataAgent
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.tools.factory import AgentToolFactory

logger = logging.getLogger(__name__)
reconfigure_logging("DEBUG")

py_module_loader.initialize()

INFO:automata.singletons.py_module_loader:Loading modules with root path: /Users/ocolegrove/automata_fresh_2/automata/core/../.. and py path: /Users/ocolegrove/automata_fresh_2/automata/core/../../automata


In [2]:
# Construct the set of all dependencies that will be used to build the tools
toolkit_list = ["py-reader"]#"context-oracle"]
tool_dependencies = dependency_factory.build_dependencies_for_tools(toolkit_list)

INFO:automata.singletons.dependency_factory:Building dependencies for toolkit_list ['py-reader']...
INFO:automata.singletons.dependency_factory:Building py_reader...
INFO:automata.singletons.dependency_factory:Creating dependency py_reader


In [3]:
# Build the tools
tools = AgentToolFactory.build_tools(toolkit_list, **tool_dependencies)

In [4]:
# Build the agent config
config_name = AgentConfigName("automata-main")

agent_config = (
    OpenAIAutomataAgentConfigBuilder.from_name(config_name)
    .with_tools(tools)
    .with_model("gpt-4")
    .with_max_iterations(2)
    .build()
)

INFO:automata.singletons.dependency_factory:Creating dependency symbol_rank
INFO:automata.singletons.dependency_factory:Creating dependency subgraph
INFO:automata.singletons.dependency_factory:Creating dependency symbol_graph
INFO:automata.symbol.graph.navigator:Pre-computing bounding boxes for all rankable symbols
Error computing bounding box for scip-python python automata 264a4fd97ce6a05c5972ef61db63b0f3802b766f `automata.code_parsers.py.ast_utils`/BoundingBox#: Module automata.code_parsers.py.ast_utils not found
Error computing bounding box for scip-python python automata 264a4fd97ce6a05c5972ef61db63b0f3802b766f `automata.code_parsers.py.ast_utils`/DocstringRemover#: Module automata.code_parsers.py.ast_utils not found
Error computing bounding box for scip-python python automata 264a4fd97ce6a05c5972ef61db63b0f3802b766f `automata.code_parsers.py.ast_utils`/DocstringRemover#visit().: Module automata.code_parsers.py.ast_utils not found
Error computing bounding box for scip-python pytho

ERROR:automata.symbol.graph.symbol_graph:Error processing scip-python python automata 264a4fd97ce6a05c5972ef61db63b0f3802b766f `automata.code_parsers.py.ast_utils`/LineItem#: Symbol(scip-python python automata 264a4fd97ce6a05c5972ef61db63b0f3802b766f `automata.code_parsers.py.ast_utils`/LineItem#, scip-python, Package(python automata 264a4fd97ce6a05c5972ef61db63b0f3802b766f), (Descriptor(automata.code_parsers.py.ast_utils, 1), Descriptor(LineItem, 2)))
ERROR:automata.symbol.graph.symbol_graph:Error processing scip-python python automata 264a4fd97ce6a05c5972ef61db63b0f3802b766f `automata.code_parsers.py.ast_utils`/fetch_bounding_box().: Symbol(scip-python python automata 264a4fd97ce6a05c5972ef61db63b0f3802b766f `automata.code_parsers.py.ast_utils`/fetch_bounding_box()., scip-python, Package(python automata 264a4fd97ce6a05c5972ef61db63b0f3802b766f), (Descriptor(automata.code_parsers.py.ast_utils, 1), Descriptor(fetch_bounding_box, 4)))
ERROR:automata.symbol.graph.symbol_graph:Error proce

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 692/692 [00:23<00:00, 29.36it/s]
INFO:automata.symbol.graph.symbol_graph:Built the rankable symbol subgraph


In [5]:
symbol_rank = dependency_factory.get("symbol_rank")
symbol_rank.get_top_symbols(
                10
            )

[('automata.core.base.database.vector.VectorDatabaseProvider',
  0.004384584761099946),
 ('automata.core.base.database.vector.ChromaVectorDatabase',
  0.0035106956754573904),
 ('automata.singletons.github_client.GitHubClient', 0.0034109238995391326),
 ('automata.core.base.database.relational.RelationalDatabase',
  0.002929687500000001),
 ('automata.llm.providers.openai.OpenAIChatCompletionProvider',
  0.0027474632488570403),
 ('automata.symbol_embedding.vector_databases.ChromaSymbolEmbeddingVectorDatabase',
  0.002706981262515736),
 ('automata.singletons.dependency_factory.DependencyFactory',
  0.002540298289368039),
 ('automata.singletons.github_client.RepositoryClient', 0.0024926875272844417),
 ('automata.llm.providers.openai.OpenAIFunction', 0.002467728584346458),
 ('automata.core.base.database.vector.JSONVectorDatabase',
  0.00240284748673057)]

In [6]:
from automata.code_parsers.py import PyReader
py_reader = PyReader()
py_reader.get_source_code("automata.core.base.database.vector", "VectorDatabaseProvider")#"core.base.database.vector.", None)

'class VectorDatabaseProvider(abc.ABC, Generic[K, V]):\n    """An abstract base class for different types of vector database providers."""\n\n    @abc.abstractmethod\n    def __len__(self) -> int:\n        pass\n\n    @abc.abstractmethod\n    def save(self) -> None:\n        """Abstract method to save data."""\n        pass\n\n    @abc.abstractmethod\n    def load(self) -> None:\n        """Abstract method to load data."""\n        pass\n\n    @abc.abstractmethod\n    def clear(self) -> None:\n        """Abstract method to clear all entries."""\n        pass\n\n    @abc.abstractmethod\n    def get_ordered_keys(self) -> List[K]:\n        """Abstract method to get all keys stored in the database."""\n        pass\n\n    @abc.abstractmethod\n    def get_ordered_embeddings(self) -> List[V]:\n        """\n        Abstract method to get an ordered list entries in the database.\n        These vectors should be ordered in the same way as the keys returned by get_ordered_keys.\n        """\n   

In [7]:
# Initialize the agent
# instructions = "Explain how OpenAI is used by the codebase. Note in particularly how it is used in embeddings and how agents consume it"
instructions = "Provide a comprehensive explanation of how SymbolRank and SymbolSearch work. Return your answer as a valid JSON string."
import textwrap
# instructions = textwrap.dedent('''
# Your task is to solve the following Github Issue

# Title:
# Specify list of supported symbols for document generation, add override flag to `EmbeddingHandler`

# Body:
# The goal is to modify the `run_doc_embedding` script to accept an arbitrary list of new symbols to regenerate [preferably in some human readable form that is parsed]. These symbols should then overwrite existing results in the database with new documentation if necessary. To implement the second step a flag will need to be added to the EmbeddingHandler.


# 1. Understand the Project: Review the Automata project to get a clear understanding of its goals and how it operates. This includes understanding how the document generation pipeline works, the role of indices and embeddings, and how they interplay to allow for efficient code and documentation generation.

# 2. Setup the Environment: Clone the Automata project into the "../repo_store/automata" directory relative to your local working directory.

#     Code: `git clone git@github.com:emrgnt-cmplxty/Automata.git ../repo_store/automata`

# 3. Generate New Indices: Once you have cloned the Automata project, navigate to the "scripts" directory and run the `generate_indices.sh` script to generate new indices.

#     Code: 
#     ```bash
#     cd scripts
#     ./generate_indices.sh
#     ```

#     You can verify the creation of the indices by navigating to the `automata-embedding-data` directory and running `git status`.

#     Code: 
#     ```bash
#     cd automata-embedding-data
#     git status
#     ```
# 4. Refresh Code Embeddings: From your main directory, run the `run-code-embedding` command. This will refresh the embeddings in the database, rolling the commit hash forward where the symbol source code hasn't changed and recalculating the index where necessary.

#     Code: `poetry run automata run-code-embedding`

# 5. Run Document Embedding: Next, run the `run-doc-embedding` command locally to see it in action.

#     Code: `poetry run automata run-doc-embedding`

#     This command generates new docs for newly added symbols and moves forward with the symbols that are in the index and map onto the database. This is a crucial part of the pipeline which generates the Automata docs.

# 6. Understand the Codebase: As you carry out these steps, ensure to take mental notes to understand the workings of the codebase. Insert print statements and other debug aids to get a sense of what's happening in the pipeline.

# 7. Modify `run_doc_embedding`: Once you are confident with your understanding of the pipeline, your goal is to modify `run_doc_embedding` to accept a list of new symbols to regenerate, and overwrite existing results in the database with new documentation if necessary.

# 8. Seek Help: If you encounter any issues or have questions, don't hesitate to ask for help. Once you have successfully carried out these tasks, check in for the next steps.

# Remember, understanding the pipeline and how everything fits together is the key to this task. 

# Begin now.
# ''')
instructions = "retrieve the source code for the VectorDatabaseProvider"
agent = OpenAIAutomataAgent(instructions, config=agent_config)

DEBUG:automata.agent.providers:Setting up agent with tools = [OpenAITool(function=<bound method PyReaderToolkitBuilder._run_indexer_retrieve_code of <automata.tools.builders.py_reader.PyReaderOpenAIToolkit object at 0x10f28a610>>, name='py-retriever-retrieve-raw-code', description='Returns the code of the python package, module, standalone function, class, or method at the given module path and sub-module (e.g. node) path. If no match is found, then "No Result Found." is returned.\n\nFor example - suppose we want the entire source code of a module located in "target_module.py" of the root directory,then the correct tool input is:\narguments: {"module_path": "target_module"}Suppose instead the file is located in a subdirectory called module_directory:\narguments: {"module_path": "module_directory.target_module"}Next, suppose that we just want to retrieve \'target_function\' in target_module:arguments: {"module_path": "module_directory.target_module", "node_path": "target_function"}Lastl

DEBUG:automata.agent.providers:
------------------------------------------------------------
Session ID: 621c3c2f-5623-45fe-8df2-0161868d39e2
------------------------------------------------------------




In [8]:
# Run the agent
result = agent.run()

DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

DEBUG:automata.llm.providers.openai:Approximately 2454 tokens were consumed prior to completion generation.


Function Call:
py-retriever-retrieve-raw-code

Arguments:
{
  "module_path": "automata.core.base.database.vector",
  "node_path": "VectorDatabaseProvider"
}

 

DEBUG:automata.llm.providers.openai:Approximately 2458 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:automata.agent.providers:Latest User Message -- 
user:
content=Execution Result:

class VectorDatabaseProvider(abc.ABC, Generic[K, V]):

    @abc.abstractmethod
    def __len__(self) -> int:
        pass

    @abc.abstractmethod
    def save(self) -> None:
        pass

    @abc.abstractmethod
    def load(self) -> None:
        pass

    @abc.abstractmethod
    def clear(self) -> None:
        pass

    @abc.abstractmethod
    def get_ordered_keys(self) -> List[K]:
        pass

    @abc.abstractmethod
    def get_ordered_embeddings(self) -> List[V]:
        pass

    @abc.abstractmethod
    def add(self, entry: V) -> None:
        pass

    @abc.abstractmethod
    def batch_add(self, entries: V) -> None:
        pass

    @abc.abstractmethod
    def

Function Call:
call_termination

Arguments:
{"result": "```python\nclass VectorDatabaseProvider(abc.ABC, Generic[K, V]):\n\n    @abc.abstractmethod\n    def __len__(self) -> int:\n        pass\n\n    @abc.abstractmethod\n    def save(self) -> None:\n        pass\n\n    @abc.abstractmethod\n    def load(self) -> None:\n        pass\n\n    @abc.abstractmethod\n    def clear(self) -> None:\n        pass\n\n    @abc.abstractmethod\n    def get_ordered_keys(self) -> List[K]:\n        pass\n\n    @abc.abstractmethod\n    def get_ordered_embeddings(self) -> List[V]:\n        pass\n\n    @abc.abstractmethod\n    def add(self, entry: V) -> None:\n        pass\n\n    @abc.abstractmethod\n    def batch_add(self, entries: V) -> None:\n        pass\n\n    @abc.abstractmethod\n    def update_entry(self, entry: V) -> None:\n        pass\n\n    @abc.abstractmethod\n    def batch_update(self, entries: List[V]) -> None:\n        pass\n\n    @abc.abstractmethod\n    def entry_to_key(self, entry: V) -> K:

DEBUG:automata.llm.providers.openai:Approximately 2838 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:automata.agent.providers:Latest User Message -- 
user:
content=Execution Result:

```python
class VectorDatabaseProvider(abc.ABC, Generic[K, V]):

    @abc.abstractmethod
    def __len__(self) -> int:
        pass

    @abc.abstractmethod
    def save(self) -> None:
        pass

    @abc.abstractmethod
    def load(self) -> None:
        pass

    @abc.abstractmethod
    def clear(self) -> None:
        pass

    @abc.abstractmethod
    def get_ordered_keys(self) -> List[K]:
        pass

    @abc.abstractmethod
    def get_ordered_embeddings(self) -> List[V]:
        pass

    @abc.abstractmethod
    def add(self, entry: V) -> None:
        pass

    @abc.abstractmethod
    def batch_add(self, entries: V) -> None:
        pass

    @abc.abstractmeth

In [9]:
# Print the result
print(f"Result:\n{result}")

Result:
Execution Result:

```python
class VectorDatabaseProvider(abc.ABC, Generic[K, V]):

    @abc.abstractmethod
    def __len__(self) -> int:
        pass

    @abc.abstractmethod
    def save(self) -> None:
        pass

    @abc.abstractmethod
    def load(self) -> None:
        pass

    @abc.abstractmethod
    def clear(self) -> None:
        pass

    @abc.abstractmethod
    def get_ordered_keys(self) -> List[K]:
        pass

    @abc.abstractmethod
    def get_ordered_embeddings(self) -> List[V]:
        pass

    @abc.abstractmethod
    def add(self, entry: V) -> None:
        pass

    @abc.abstractmethod
    def batch_add(self, entries: V) -> None:
        pass

    @abc.abstractmethod
    def update_entry(self, entry: V) -> None:
        pass

    @abc.abstractmethod
    def batch_update(self, entries: List[V]) -> None:
        pass

    @abc.abstractmethod
    def entry_to_key(self, entry: V) -> K:
        pass

    @abc.abstractmethod
    def contains(self, key: K) -> 

In [17]:
x = result.replace("Execution Result:","").replace("```python","").replace("```","").strip()
x

'class VectorDatabaseProvider(abc.ABC, Generic[K, V]):\n\n    @abc.abstractmethod\n    def __len__(self) -> int:\n        pass\n\n    @abc.abstractmethod\n    def save(self) -> None:\n        pass\n\n    @abc.abstractmethod\n    def load(self) -> None:\n        pass\n\n    @abc.abstractmethod\n    def clear(self) -> None:\n        pass\n\n    @abc.abstractmethod\n    def get_ordered_keys(self) -> List[K]:\n        pass\n\n    @abc.abstractmethod\n    def get_ordered_embeddings(self) -> List[V]:\n        pass\n\n    @abc.abstractmethod\n    def add(self, entry: V) -> None:\n        pass\n\n    @abc.abstractmethod\n    def batch_add(self, entries: V) -> None:\n        pass\n\n    @abc.abstractmethod\n    def update_entry(self, entry: V) -> None:\n        pass\n\n    @abc.abstractmethod\n    def batch_update(self, entries: List[V]) -> None:\n        pass\n\n    @abc.abstractmethod\n    def entry_to_key(self, entry: V) -> K:\n        pass\n\n    @abc.abstractmethod\n    def contains(self, 

In [18]:
y = py_reader.get_source_code("automata.core.base.database.vector", "VectorDatabaseProvider")
y

'class VectorDatabaseProvider(abc.ABC, Generic[K, V]):\n    """An abstract base class for different types of vector database providers."""\n\n    @abc.abstractmethod\n    def __len__(self) -> int:\n        pass\n\n    @abc.abstractmethod\n    def save(self) -> None:\n        """Abstract method to save data."""\n        pass\n\n    @abc.abstractmethod\n    def load(self) -> None:\n        """Abstract method to load data."""\n        pass\n\n    @abc.abstractmethod\n    def clear(self) -> None:\n        """Abstract method to clear all entries."""\n        pass\n\n    @abc.abstractmethod\n    def get_ordered_keys(self) -> List[K]:\n        """Abstract method to get all keys stored in the database."""\n        pass\n\n    @abc.abstractmethod\n    def get_ordered_embeddings(self) -> List[V]:\n        """\n        Abstract method to get an ordered list entries in the database.\n        These vectors should be ordered in the same way as the keys returned by get_ordered_keys.\n        """\n   

In [19]:
x==y

False